# SAMWISE Moment Of Inertia Estimation

In [1]:
import numpy as np

In [2]:
class RectangularPrism:
    def __init__(self, 
                 com: np.array,
                 size: np.array,
                 mass: float,
                 R=np.eye(3)) -> None:
        self.com = com   # com in CAD frame [m]
        self.R = R       # rotation matrix mapping local frame to CAD frame (columns are local axes in CAD frame)
        self.size = size # (x, y, z) size in local frame [m]
        self.mass = mass # mass [kg]
        
        assert np.abs(np.linalg.det(R) - 1) < 1e-10 # Rotation matrix should have determinant 1

    @property
    def I_com(self) -> np.array:
        # Moment of inertia about center of mass in local frame [kg m^2]
        # https://dynref.engr.illinois.edu/rem.html
        dx = self.size[0]
        dy = self.size[1]
        dz = self.size[2]

        Ixx = self.mass * (dy**2 + dz**2) / 12
        Iyy = self.mass * (dx**2 + dz**2) / 12
        Izz = self.mass * (dx**2 + dy**2) / 12

        return np.array([
            [Ixx, 0, 0],
            [0, Iyy, 0],
            [0, 0, Izz]
        ])
    
    @property
    def I_com_cad_frame(self) -> np.array:
        # Moment of inertia about the center of mass, in the cad frame [kg m^2]
        # TODO Be 100% sure this works
        return self.R @ self.I_com @ self.R.T

    def I_about_point(self, center: np.array) -> np.array:
        # Return the moment of inertia in the cad frame about a given pivot [kg m^2]
        x, y, z = self.com - center
        I_displacement = self.mass * np.array([
            [y**2 + z**2, -x * y, -x * z],
            [-x * y, x**2 + z**2, -y * z],
            [-x * z, -y * z, x**2 + y**2],
        ])
        return self.I_com_cad_frame + I_displacement

In [10]:
def get_inertial_properties(bodies: list[RectangularPrism]):
    m_overall = sum([body.mass for body in bodies])
    com_overall = sum([body.com * body.mass for body in bodies]) / m_overall

    I_overall = sum([body.I_about_point(com_overall) for body in bodies])

    values, vectors = np.linalg.eig(I_overall)

    return m_overall, com_overall, I_overall


def print_inertial_properties(m, com, I):
    print("\n\n----------INERTIAL PROPERTIES------------")
    print(f"Total mass: {m} [g]")
    print(f"Com in cad frame: {com} [mm]")

    values, vectors = np.linalg.eig(I)

    print('')
    print("Principle moments of inertia about center of mass [g mm^2 * 10^6]")
    print(f'{values / 1e6}')

    print('')
    print("Principle axes")
    print(f'{vectors}')

    print('')
    print("Moment of inertia about center of mass [g mm^2 * 10^6]")
    print(I / 1e6)


In [15]:
def inertia_from_wheel_mass(wheel_mass: float):

    x = np.sqrt(2) / 2

    bodies: list[RectangularPrism] = [
        # +X Panel 
        RectangularPrism(
            com=np.array([50, 0, 0]),
            size=np.array([2.5, 100, 215]),
            mass=300, # TODO
        ),

        # -X Panel 
        RectangularPrism(
            com=np.array([-50, 0, 0]),
            size=np.array([2.5, 100, 215]),
            mass=300, # TODO
        ),
        
        # +Y Panel 
        RectangularPrism(
            com=np.array([0, 50, 0]),
            size=np.array([100, 2.5, 215]),
            mass=150,
        ),

        # -Y Panel 
        RectangularPrism(
            com=np.array([0, -50, 0]),
            size=np.array([100, 2.5, 215]),
            mass=150,
        ),
        # +X Solar Panel
        RectangularPrism(
            com=np.array([50+85*x, -50-85*x, 0]),
            size=np.array([170, 5, 215]),
            mass=500, # TODO
            R = np.array([
                [x, x, 0],
                [-x, x, 0],
                [0, 0, 1]
            ]),
        ),

        # -X Solar Panel
        RectangularPrism(
            com=np.array([-50-85*x, 50+85*x, 0]),
            size=np.array([170, 5, 215]),
            mass=500, # TODO
            R = np.array([
                [x, x, 0],
                [-x, x, 0],
                [0, 0, 1]
            ]),
        ),
        # +Z Panel 
        RectangularPrism(
            com=np.array([0, 0, 215/2]),
            size=np.array([100, 100, 5]),
            mass=100,
        ),
        # -Z Panel
        RectangularPrism(
            com=np.array([0, 0, -215/2]),
            size=np.array([100, 100, 5]),
            mass=100,
        ),

        # BOARD STACKS
        # Raspberry Pi Unit
        RectangularPrism(
            com=np.array([0, 10, -215/2 + 32.5]),
            size=np.array([90, 60, 25]),
            mass=90,
        ),
        # Batteries 
        RectangularPrism(
            com=np.array([0, 0, 215/2  -50 -75/2]),
            size=np.array([90, 90, 75]),
            mass=1000,
        ),
        # PiCubed 
        RectangularPrism(
            com=np.array([0, 0, 215/2  - 45]),
            size=np.array([100, 100, 5]),
            mass=50,
        ),

        # ADCS Board
        RectangularPrism(
            com=np.array([0, 0, -215/2 + 75]),
            size=np.array([100, 100, 30]),
            mass=100,
        ),

        # Reaction Wheels
        RectangularPrism(
            com=np.array([0, 0, -215/2 + 75]),
            size=np.array([100, 100, 30]),
            mass=4 * wheel_mass,
        ),
    ]

    return get_inertial_properties(bodies)


values = inertia_from_wheel_mass(50)
print_inertial_properties(*values)




----------INERTIAL PROPERTIES------------
Total mass: 3540 [g]
Com in cad frame: [0.         0.25423729 1.87146893] [mm]

Principle moments of inertia about center of mass [g mm^2 * 10^6]
[14.21389908 40.87154478 32.01974461]

Principle axes
[[-0.71403148 -0.70008916 -0.00584955]
 [ 0.70010831 -0.71403363 -0.00208051]
 [-0.00272024 -0.00558087  0.99998073]]

Moment of inertia about center of mass [g mm^2 * 10^6]
[[27.28008243 13.32603264  0.        ]
 [13.32603264 27.80521749  0.06918432]
 [ 0.          0.06918432 32.01988855]]


In [ ]:
wheel_masses = [
    6.9,
    20.1,
    20.1,
    28.9,
    57.9,
    127,
]

wheel_names = [
    "Series 1509 B  (006 B, 012 B)",
    "Series 2610 B (006 B, 012 B)",
    "Series 2610 B 006 SC",
    "Series 2214 BXT 006 H",
    "Series 3216 BXT H SC",
    "Series 4221 BXT H SC (018, 024, 048)",
]

I_values = [inertia_from_wheel_mass(m)[2] for m in wheel_masses]
I_principals = [np.linalg.eig(I)[0] for I in I_values]

print("                                           Wheel mass [g]          I_min [kg m^2]          I_max [g mm^2]")
for i in range(len(wheel_masses)):
    print(f"{wheel_names[i]:<40}    {wheel_masses[i]:.4f},                 {min(I_principals[i]) / 1e9:.4f},                   {max(I_principals[i]) / 1e9:.4f}")

                                           Wheel mass [g]          I_min [kg m^2]          I_max [g mm^2]
Series 1509 B  (006 B, 012 B)               6.900,                 0.014,                    0.041
Series 2610 B (006 B, 012 B)                20.100,                 0.014,                    0.041
Series 2610 B 006 SC                        20.100,                 0.014,                    0.041
Series 2214 BXT 006 H                       28.900,                 0.014,                    0.041
Series 3216 BXT H SC                        57.900,                 0.014,                    0.041
Series 4221 BXT H SC (018, 024, 048)        127.000,                 0.015,                    0.041
